In [1]:
import numpy as np

In [2]:
from lsst.sims.catalogs.generation.db import CatalogDBObject
from lsst.sims.catalogs.generation.db import CatalogDBObject

In [3]:
import lsst.sims.catUtils.baseCatalogModels as bcm
from lsst.sims.catUtils.baseCatalogModels import GalaxyTileObj
from lsst.sims.catalogs.measures.instance import InstanceCatalog
from lsst.sims.utils import ObservationMetaData



/usr/local/manual/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
Duplicate object type id 25 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want

In [4]:
from lsst.sims.catUtils.baseCatalogModels import SNObj

In [5]:
s  = SNObj()

In [6]:
names = s.get_column_names()

/usr/local/manual/anaconda/lib/python2.7/site-packages/sqlalchemy/dialects/mssql/base.py:1294: SAWarning: Did not recognize type 'geography' of column 'geopoint'
  return fn(*arg, **kw)
/usr/local/manual/anaconda/lib/python2.7/site-packages/sqlalchemy/dialects/mssql/base.py:1294: SAWarning: Did not recognize type 'geography' of column 'geocenter'
  return fn(*arg, **kw)
/usr/local/manual/anaconda/lib/python2.7/site-packages/sqlalchemy/dialects/mssql/base.py:1294: SAWarning: Did not recognize type 'geography' of column 'geobox'
  return fn(*arg, **kw)


In [7]:
gTile = GalaxyTileObj()
class GTileCopy(InstanceCatalog):
    column_outputs = ['raJ2000', 'decJ2000', 'galtileid','redshift']
    override_formats = {'raJ2000': '%8e', 'decJ2000': '%8e'}

In [8]:
for key in names:
    if 'snra' in names[key]:
        print key

TwinkSN


In [9]:
print names['TwinkSN']

['galtileid', 'htmid', 'snid', 't0', 'c', 'x1', 'x0', 'redshift', 'snra', 'sndec', 'cx', 'cy', 'cz']


In [10]:
class SNCopy(InstanceCatalog):
    
    column_outputs = ['raJ2000', 'decJ2000', 'snid', 'galtileid','t0', 'x0', 'x1', 'c', 'redshift', ]
    override_formats = {'raJ2000': '%8e', 'decJ2000': '%8e', 'x0':'%8e'}
                        #'x0':'%8e', 'x1':'%8e', 'c':'%8e', 't0':'%8e'}

In [11]:
degConv = np.array([1., 1./60., 1./3600.])
raConv = degConv / 24.0 * 360.
centralRA = np.dot(np.array([3., 32., 30]), raConv) #03h 32m 30s
centralDec = np.dot(np.array([-28, 6., 0.]), degConv)
patchRadius = 0.4 * np.sqrt(2) #np.dot(np.array([0.0, 10.0, 0.]), degConv)

In [12]:
TwinklesObsMetaDataSmall = ObservationMetaData(boundType='box',pointingRA=centralRA,pointingDec=centralDec,
                                          boundLength=0.167, mjd=49540.0)

In [13]:
tsmall = SNCopy(db_obj=s, obs_metadata=TwinklesObsMetaDataSmall)

In [14]:
tsmall.write_catalog('TwinklesSN.dat')

In [15]:
!wc -l TwinklesSN.dat

   72780 TwinklesSN.dat


In [16]:
!head TwinklesSN.dat

#raJ2000, decJ2000, snid, galtileid, t0, x0, x1, c, redshift
9.268217e-01, -4.856822e-01, 60010597975, 60010597975, 3538.2554, 7.806233e-06, -0.3295, -0.0126, 0.5919
9.268106e-01, -4.856701e-01, 60017065964, 60017065964, 3627.1300, 1.801546e-05, -1.1427, 0.0184, 0.3758
9.267375e-01, -4.855998e-01, 60001349751, 60001349751, 2789.8328, 4.783149e-06, 0.3052, -0.0837, 0.7626
9.267382e-01, -4.855999e-01, 10000060001349751, 60001349751, 2453.3036, 3.593970e-06, 0.6973, -0.0185, 0.7626
9.267383e-01, -4.856026e-01, 40000060001349751, 60001349751, 352.9995, 3.963334e-06, -0.6465, -0.0434, 0.7626
9.267400e-01, -4.856007e-01, 30000060001349751, 60001349751, 1785.3676, 3.382491e-06, 1.2164, 0.1182, 0.7626
9.267414e-01, -4.856025e-01, 20000060001349751, 60001349751, 619.4357, 8.153988e-06, 0.1365, -0.1729, 0.7626
9.267017e-01, -4.856932e-01, 60001948889, 60001948889, 72.4902, 7.510870e-06, 0.8098, 0.0876, 0.5621
9.267012e-01, -4.856931e-01, 10000060001948889, 60001948889, 2443.7046, 4.451357e-06, -

In [18]:
from lsst.sims.catUtils.mixins import TSNIaCatalog

In [19]:
ts = TSNIaCatalog(db_obj=s, obs_metadata=TwinklesObsMetaDataSmall)

RuntimeError: maximum recursion depth exceeded

In [ ]:
class SNCat()

In [21]:
np.radians(53.1029753341)

0.92682176218538093

In [25]:
gsmall = GTileCopy(db_obj=gTile, obs_metadata=TwinklesObsMetaDataSmall)

In [26]:
gsmall.write_catalog('gsmall.cat')

In [27]:
9.268184e-01

0.016176032648065813

In [28]:
np.radians(53.1027826951)

0.92681840000056304